# Import required modules
Make sure you installed the conda environment and activated it first by running:

```
conda env create -f env.yml
```

Then you can choose that environment for the jupyter kernel.

## Important!!
If you want to use the witwidget and haven't already run these commands you must do so!

```
jupyter nbextension install --py --symlink --sys-prefix witwidget
jupyter nbextension enable --py --sys-prefix witwidget
```

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import linear_model, metrics, model_selection
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

# What-If Tool Helper Functions

In [2]:
def df_to_examples(df, columns=None):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if row[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif row[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            else:
                raise TypeError("Oh No!")
        examples.append(example)
    return examples

# Load data and make required directories

In [3]:
dat_in_pth = "../data/features.pkl"
dat_df = pd.read_pickle(dat_in_pth)

results_dir = "results"
seed = 8675309
np.random.seed(seed)

if not os.path.exists(results_dir):
    os.makedirs(results_dir)

training_set_tss_ids_pth = results_dir + "/training_set_tss_ids.txt"
heldout_test_set_tss_ids_pth = results_dir + "/heldout_test_set_tss_ids.txt"
crossval_results_pth = results_dir + "/crossval_results.csv"
test_performance_pth = results_dir + "/heldout_test_performance.csv"
coefs_pth = results_dir + "/coefficients.csv"

dat_df.head()

,M0004_1.02_FWD_1,M0004_1.02_FWD_2,M0004_1.02_FWD_3,M0004_1.02_FWD_4,M0004_1.02_FWD_5,M0004_1.02_FWD_6,M0004_1.02_FWD_7,M0005_1.02_FWD_1,M0005_1.02_FWD_2,M0005_1.02_FWD_3,...,GA_REV_2,GA_REV_3,GA_REV_4,GA_REV_5,GA_REV_6,GA_REV_7,GCcontent,CAcontent,GAcontent,class
index,,,,,,,,,,,,,,,,,,,,,
Chr4_+_7750398_AT4G13310_root_0,1.440441,0.000000,0.000000,4.508889,9.438149,9.714371,17.533581,5.501110,0.0,0.0,...,11.805773,9.450124,12.967102,8.974781,14.424215,13.597577,0.300,0.590,0.580,1
Chr4_+_11302680_AT4G21200_leaf_0,11.111250,11.111250,0.000000,0.000000,12.846526,16.472290,5.290287,2.406347,0.0,0.0,...,0.824729,9.927819,16.496307,6.568488,4.865940,4.865940,0.340,0.540,0.430,1
Chr4_-_10918262_AT4G20210_root_0,0.293377,4.510425,4.217049,4.559656,5.156968,0.597312,0.000000,0.000000,0.0,0.0,...,2.222053,11.242960,9.343733,6.638287,25.419486,18.781199,0.410,0.545,0.475,1
Chr5_-_26757236_AT5G67030_tair_0,0.555420,2.275065,2.275065,0.000000,0.000000,0.189317,22.804014,0.000000,0.0,0.0,...,9.204328,3.095468,0.000000,11.282284,13.337318,11.061596,0.425,0.495,0.440,1
Chr1_-_25046860_AT1G67080_tair_0,0.000000,1.398959,1.398959,1.377300,2.470973,1.093673,0.000000,0.000000,0.0,0.0,...,11.599996,10.826384,11.891774,11.529227,11.948718,8.528950,0.305,0.545,0.550,1


# Split data into train/test sets

In [76]:
x_train, x_heldout, y_train, y_heldout = model_selection.train_test_split(
    dat_df.iloc[:, :-1],  
    dat_df.iloc[:, -1],
    test_size=0.2,
    stratify=dat_df["class"]
)

### Optionally save IDs

In [ ]:
np.savetxt(training_set_tss_ids_pth, x_train.index.values, fmt="%s")
np.savetxt(heldout_test_set_tss_ids_pth, x_heldout.index.values, fmt="%s")

### Convert to tensors

In [77]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float64)
x_heldout = tf.convert_to_tensor(x_heldout, dtype=tf.float64)
y_train = tf.one_hot(y_train, 2, dtype=tf.float64)
y_heldout = tf.one_hot(y_heldout, 2, dtype=tf.float64)

# Define model object

In [69]:
n_classes = 2
n_batches = 1000
learning_rate = 0.01
batch_size = 128

dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(x_train.shape[0])
    .batch(batch_size)
)

# Initialize weights and biases with values from Gaussians
weights = tf.Variable(tf.random.normal(shape=(x_train.shape[1], n_classes), dtype=tf.float64))
biases = tf.Variable(tf.random.normal(shape=(n_classes,), dtype=tf.float64))

optimizer = tf.optimizers.SGD(learning_rate)

# Fit the model on training data

In [72]:
def accuracy(y_pred, y_true):
    # Predicted class is the index of the highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

for batch_num, (batch_xs, batch_ys) in enumerate(dataset.take(n_batches), 1):
    with tf.GradientTape() as tape:
        y_pred = tf.sigmoid(tf.matmul(batch_xs, weights) + biases)
        loss = tf.nn.sigmoid_cross_entropy_with_logits(batch_ys, y_pred)
    gradients = tape.gradient(loss, [weights, biases])
    optimizer.apply_gradients(zip(gradients, [weights, biases]))

    if batch_num % display_step == 0:
        y_pred = tf.sigmoid(tf.matmul(batch_xs, weights) + biases)
        loss = tf.nn.sigmoid_cross_entropy_with_logits(batch_ys, y_pred)
        acc = accuracy(batch_ys, y_pred)
        print(f"Batch number: {batch_num}, loss: {loss}, accuracy: {acc}")

NameError: name 'cross_entropy' is not defined

# Test the model

In [7]:
y_true, y_prediction_probabilities = (
    heldout_test["class"],
    log_cv.predict_proba(heldout_test.iloc[:, :-1]),
)

# What-If Tool

In [8]:
num_datapoints = 10
tool_height_in_px = 500

### Important
The WiT expects tensorflow objects, so the code cell below is required.

In [9]:
train_df = dat_df.iloc[:, :10]
test_df = heldout_test.iloc[:, :10]
test_examples = df_to_examples(test_df)

In [ ]:
config_builder = (
    WitConfigBuilder(test_examples)
#     .set_custom_predict_fn(log_cv.predict)
#     .set_label_vocab(['Negative', 'Positive'])
)
WitWidget(config_builder, height=tool_height_in_px)

# Compute performance metrics for predictions on the test set

In [7]:
roc_curve_df = (
    pd.DataFrame(metrics.roc_curve(y_true, y_prediction_probabilities[:, 1]))
    .T.rename(columns={0: "FPR", 1: "TPR", 2: "threshold"})
    .drop(index=0)
    .reset_index(drop=True)
)
roc_curve_df["Y"] = roc_curve_df["TPR"] - roc_curve_df["FPR"]
youden_T = roc_curve_df.iloc[roc_curve_df["Y"].idxmax()]["threshold"]

y_pred_youden = np.where(y_prediction_probabilities[:, 1] >= youden_T, 1, 0)

heldout_perf_roc = metrics.roc_auc_score(y_true, y_prediction_probabilities[:, 1])
heldout_perf_prc = metrics.average_precision_score(
    y_true, y_prediction_probabilities[:, 1]
)

confusion_mat = metrics.confusion_matrix(y_true, y_pred_youden)
TN = confusion_mat[0][0]
FP = confusion_mat[0][1]
FN = confusion_mat[1][0]
TP = confusion_mat[1][1]

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP / (TP + FN)
# Specificity or true negative rate
TNR = TN / (TN + FP)
# Precision or positive predictive value
PPV = TP / (TP + FP)

heldout_perf_f1 = 2 * ((PPV * TPR) / (PPV + TPR))

perf_metrics = pd.Series({
    "C": log_cv.C_[0],
    "auROC": heldout_perf_roc,
    "auPRC": heldout_perf_prc,
    "youden_T": youden_T,
    "sensitivity": TPR,
    "specificity": TNR,
    "precision": PPV,
    "f1": heldout_perf_f1,
    "seed": seed,
})
perf_metrics

C              4.750810e+03
auROC          6.952663e-01
auPRC          6.470624e-01
youden_T       5.342641e-01
sensitivity    7.307692e-01
specificity    6.538462e-01
precision      6.785714e-01
f1             7.037037e-01
seed           8.675309e+06
dtype: float64

In [8]:
coefficients = pd.DataFrame(log_cv.coef_, columns=dat_df.columns[:-1])
coefficients

,M0004_1.02_FWD_1,M0004_1.02_FWD_2,M0004_1.02_FWD_3,M0004_1.02_FWD_4,M0004_1.02_FWD_5,M0004_1.02_FWD_6,M0004_1.02_FWD_7,M0005_1.02_FWD_1,M0005_1.02_FWD_2,M0005_1.02_FWD_3,...,GA_REV_1,GA_REV_2,GA_REV_3,GA_REV_4,GA_REV_5,GA_REV_6,GA_REV_7,GCcontent,CAcontent,GAcontent
0,0.000448,0.0,-0.007917,-0.001143,0.004216,-0.007482,-0.004137,0.0,-0.000461,-0.039019,...,0.004723,-0.002895,-0.00155,0.002511,0.004984,0.006744,0.002118,0.0,0.0,0.0


# Save files

In [9]:
# save metrics
perf_metrics.to_csv(test_performance_pth, header=True)

# save coefficient values
coefficients.to_csv(coefs_pth, header=True)